# 앙상블 실습

수업 시간에 test data에 대한 예측 정확도를 높이는 방법으로 앙상블이 사용된다고 설명한 바 있다. 앙상블을 한 마디로 표현하면 **집단 지성**으로, 여러 가지 모델을 결합해 최종 예측을 진행한다. 주로 variance를 줄이는 데 큰 역할을 하는 앙상블은 대부분의 task에서 성능이 매우 뛰어난 방법론이라 알려져 있다. 

### 합의 기반 결합

앙상블 방법론은 머신 러닝 모델의 결합 방법에 따라 합의 기반 결합과 학습 기반 결합으로 나눌 수 있다. 이번 실습에서는 합의 기반 결합에 대해 알아보자.

![](https://miro.medium.com/max/700/1*I7NsQXwyR36XK62s1iDNzQ.png)

합의 기반 결합은 쉽게 말해 **다수결**을 이용하는 것이다. 단순한 예로, 70%의 정확도를 가지는 모델이 5개 있다고 하자. 만약 이들이 서로 독립이라면, 이들의 결과로 다수결 투표를 진행했을 때의 정확도는 83.7% 에 달한다.

이 때 중요한 것은 모델들이 서로 **독립**이어야 한다는 것이다. 독립이 아니라 똑같은 모델 5개이면 다수결을 해도 정확도는 70 %에서 변하지 않는다. 따라서 합의 기반 결합에서는 서로 독립인, 혹은 다양한 특징을 갖는 모델들을 생성해야 한다는 것이 매우 중요하다.  

### 합의 기반 결합 - Voting

Voting 기법은 말 그대로 여러 개의 모델에게 정답이 무엇인지 투표시키는 방식을 말한다. 이 때 각 모델은 서로 다른 방법론을 통해 만들어진다. python의 scikit-learn 패키지의 [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html?highlight=votingclassifier#sklearn.ensemble.VotingClassifier) 를 이용해 기본적인 voting 방법론을 직접 실행해보자.

[Covertype](https://archive.ics.uci.edu/ml/datasets/Covertype) 데이터셋을 사용해 보자. Scikit-learn의 [fetch_covtype](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype) 함수로 데이터를 받아올 수 있다.

In [1]:
from sklearn.datasets import fetch_covtype
x, y = fetch_covtype(return_X_y=True)
print(x.shape)
print(y.shape)

(581012, 54)
(581012,)


In [2]:
from sklearn.model_selection import train_test_split

# 데이터 나누기
num_use_data = 2000
x = x[:num_use_data, :]
y = y[:num_use_data]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

print(x_train.shape, x_test.shape)

(1600, 54) (400, 54)


VotingClassifier를 학습하기 위해, 수업에서 배웠던 세 가지 분류기들을 이용해 보자. 먼저 필요한 모델들을 import 한 후 각각 생성하자.

In [3]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [4]:
# 모델 객체 생성하기
log_clf = LogisticRegression()
dt_clf = DecisionTreeClassifier(random_state=1)
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), 
                ('dt', dt_clf), 
                ('svc', svm_clf)])

각 모델들을 학습시킨 후 정확도를 비교해 보자.

In [5]:
# 각 모델 학습 및 정확도 확인하기
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

voting_clf.fit(x_train, y_train)
pred_test = voting_clf.predict(x_test)
print(f'voting accuracy: {accuracy_score(y_test, pred_test)}')

for name, estimator in voting_clf.named_estimators_.items():
  estimator.fit(x_train, y_train)
  pred_test_estim = estimator.predict(x_test)
  print(f'{name}: {accuracy_score(y_test, pred_test_estim)}')

voting accuracy: 0.69
lr: 0.615
dt: 0.7475
svc: 0.605


VotingClassifier의 파라미터 중 하나인 voting은 다수결의 방식을 뜻한다. Default인 'hard'는 각 모델이 표를 하나씩 행사하는 방식이며, 'soft'는 각 모델의 분류 확률을 모두 더해 가장 높은 확률로 예측하는 방식이다. 일반적으로 soft voting 방식이 더 효과가 좋다고 알려져 있다. 이를 구현하고 위의 hard voting 방식과 비교해 보자.

In [15]:
# TODO: soft voting 방식으로 모델 생성하기
log_clf_soft = LogisticRegression()
dt_clf_soft = DecisionTreeClassifier(random_state=1)
svm_clf_soft = SVC(probability=True)

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf_soft), 
                ('dt', dt_clf_soft), 
                ('svc', svm_clf_soft)],
    voting = 'soft'
    )

In [16]:
# TODO: soft voting 방식으로 생성한 모델 학습하기
voting_clf_soft.fit(x_train, y_train)
pred_test = voting_clf_soft.predict(x_test)
print(f'voting accuracy: {accuracy_score(y_test, pred_test)}')

for name, estimator in voting_clf_soft.named_estimators_.items():
  estimator.fit(x_train, y_train)
  pred_test_estim = estimator.predict(x_test)
  print(f'{name}: {accuracy_score(y_test, pred_test_estim)}')

voting accuracy: 0.765
lr: 0.615
dt: 0.7475
svc: 0.605


### 합의 기반 결합 - Bagging

Bagging 기법은 voting과 달리 하나의 머신 러닝 방법론을 사용하지만, 주어진 학습 데이터나 변수를 모두 사용하지 않고 임의로 추출하여 다양한 모델을 학습시킨 후 이들의 합의 기반 결합을 진행한다. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Ensemble_Bagging.svg/512px-Ensemble_Bagging.svg.png)

Bagging의 종류는 각 모델의 학습용 데이터를 어떻게 구성하느냐에 따라 아래의 네 가지로 구분지을 수 있다.


* Pasting: 같은 데이터를 중복해서 추출하지 않음
* Bagging: 같은 데이터 샘플을 중복해서 추출
* Random Subspace: 데이터가 아니라 주어진 독립 변수 중 일부를 랜덤 추출
* Random Patches: 데이터와 독립 변수 모두 랜덤 추출해서 사용





![alt text](https://datascienceschool.net/upfiles/677f18a151c64e5daa9c28f6cb564808.png)

먼저 일반적인 의사결정 나무 분류기를 학습한 후, [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html?highlight=bagging#sklearn.ensemble.BaggingClassifier)를 이용해 bagging을 적용한 의사결정 나무 분류기를 학습해 보자.

In [17]:
tree_clf = DecisionTreeClassifier(random_state=1)
tree_clf.fit(x_train, y_train)
pred_test = tree_clf.predict(x_test)
print(f'accuracy: {accuracy_score(y_test, pred_test)}')

accuracy: 0.7475


In [18]:
from sklearn.ensemble import BaggingClassifier

In [21]:
# TODO: BaggingClassifier를 사용해 의사결정 나무 분류기 학습 및 정확도 계산하기
bagging_clf = BaggingClassifier(base_estimator=tree_clf,
                        n_estimators=10, random_state=0).fit(x_train, y_train)

pred_test = bagging_clf.predict(x_test)
print(f'accuracy: {accuracy_score(y_test, pred_test)}')

accuracy: 0.865


의사결정 나무에서 배운 재귀적 분기에 대해 떠올려보자. 분기를 할 후보들을 각 독립 변수들로 정렬한 후에 최적의 분기점을 불순도 함수를 이용해 찾는다고 배웠다. 이 때, 모든 독립 변수를 사용하는 것이 아니라 **일부**만을 골라 분기 후보를 찾는다면, 더 빠른 계산이 가능하다. 이것이 바로 위에서 말한 random subspace 혹은 random patches의 개념을 적용한 것이다.

즉, random patches를 의사결정 나무에 사용한 것이 바로 RandomForest 이다. Scikit-learn 에서는 [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforest#sklearn.ensemble.RandomForestClassifier)에서 이 모델을 제공하고 있으니, 이를 사용해서 분류해 보자.

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
# TODO: RandomForestClassifier를 사용해 분류기 학습 및 정확도 계산하기
rf_clf = RandomForestClassifier(random_state=1)
rf_clf.fit(x_train, y_train)
pred_test = rf_clf.predict(x_test)
print(f'accuracy: {accuracy_score(y_test, pred_test)}')

bagging_clf = BaggingClassifier(base_estimator=rf_clf,
                        n_estimators=100, random_state=0).fit(x_train, y_train)

pred_test = bagging_clf.predict(x_test)
print(f'bagged accuracy: {accuracy_score(y_test, pred_test)}')

accuracy: 0.83
bagged accuracy: 0.8425


GridSearchCV를 이용해 최고의 파라미터 조합을 찾아보자.

In [26]:
from sklearn.model_selection import GridSearchCV

In [30]:
# TODO: GridSearchCV를 이용해 최적의 조합을 찾고 그 때의 정확도 계산하기
param_grid2 = {'criterion': ['gini','entropy'], 
              'max_depth': [50, 100, 300],
               'min_samples_splitint': [2,10,50]}

bagging_clf_grid = GridSearchCV(estimator=bagging_clf, param_grid=param_grid2, cv=4, scoring='accuracy')
bagging_clf_grid.fit(x_train, y_train)

print(bagging_clf_grid.best_params_)
print(bagging_clf_grid.best_score_)

ValueError: ignored